# Load as LUME-model

Loads the base neural network model, the `sim_to_nn` transformers and the simulation variable specification from their respective files to create a [LUME-model](https://github.com/slaclab/lume-model/). The resulting instance of `TorchModel` enforces requirements on the input and output variables and can be wrapped in a `TorchModule`. The `TorchModule` can be used like a `torch.nn.Module` and is tested on a small set of simulation data.

Here we show how to register and update this model with MLflow.

In [1]:
import os
import sys
import torch
import mlflow
import matplotlib.pyplot as plt

from lume_model.utils import variables_from_yaml
from lume_model.models import TorchModel, TorchModule

from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")  
#os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:8082" # for local testing

In [2]:
mlflow.set_experiment("lcls-injector-ML")

<Experiment: artifact_location='mlflow-artifacts:/742261476526561220', creation_time=1752699169877, experiment_id='742261476526561220', last_update_time=1752699169877, lifecycle_stage='active', name='lcls-injector-ML', tags={}>

In [3]:
# Ensure no active run is lingering
if mlflow.active_run():
    mlflow.end_run()

## Load the model as TorchModel and TorchModule

In [4]:
# load transformers
input_sim_to_nn = torch.load("../model/input_sim_to_nn.pt", weights_only=False)
output_sim_to_nn = torch.load("../model/output_sim_to_nn.pt", weights_only=False)
# load in- and output variable specification
input_variables, output_variables = variables_from_yaml("../model/sim_variables.yml")

# create TorchModel
lume_model = TorchModel(
    model="../model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_sim_to_nn],
    output_transformers=[output_sim_to_nn],
)

# wrap in TorchModule
lume_module = TorchModule(
    model=lume_model,
    input_order=lume_model.input_names,
    output_order=lume_model.output_names,
)

Loaded PyTorch model from file: ../model/model.pt


In [5]:
# load example data that we need for mlflow
inputs_small = torch.load("../info/inputs_small.pt")
outputs_small = torch.load("../info/outputs_small.pt")

# Register model to MLflow

See function signature for reference:

In [6]:
lume_module.register_to_mlflow?

Signature:
lume_module.register_to_mlflow(
    artifact_path: str,
    registered_model_name: str | None = None,
    tags: dict[str, typing.Any] | None = None,
    version_tags: dict[str, typing.Any] | None = None,
    alias: str | None = None,
    run_name: str | None = None,
    log_model_dump: bool = True,
    save_jit: bool = False,
    **kwargs,
)
Docstring:
Registers the model to MLflow if mlflow is installed. Each time this function is called, a new version
of the model is created. The model is saved to the tracking server or local directory, depending on the
MLFLOW_TRACKING_URI.

If no tracking server is set up, data and artifacts are saved directly under your current directory. To set up
a tracking server, set the environment variable MLFLOW_TRACKING_URI, e.g. a local port/path. See
https://mlflow.org/docs/latest/getting-started/intro-quickstart/ for more info.

Args:
    artifact_path: Path to store the model in MLflow.
    registered_model_name: Name of the registered model 

In [7]:
model_info = lume_module.register_to_mlflow(
            artifact_path="lcls_injector_torch_module",
            registered_model_name="lcls_injector_torch_module", # not necessary but required for adding tags/aliases
            tags={"type":"surrogate_injector"}, # example tag, if desired
            run_name="lume-test" # will be generated randomly if not provided
)

🏃 View run lume-test at: http://127.0.0.1:8082/#/experiments/742261476526561220/runs/ce536dd7f16e48e69da0bd5ae8f20dbd
🧪 View experiment at: http://127.0.0.1:8082/#/experiments/742261476526561220


Successfully registered model 'lcls_injector_torch_module'.
Created version '1' of model 'lcls_injector_torch_module'.


When calling `lume_module.register_to_mlflow` again with the same `registered_model_name`, the model version will be incremented.

# Predict using loaded model

In [8]:
# original model
with torch.no_grad():
    predictions = lume_module(inputs_small)

# Load using version we just registered to MLflow
version = model_info.registered_model_version
model_saved = f"models:/lcls_injector_torch_module/{version}"
model_saved = mlflow.pytorch.load_model(model_saved)
predictions_load = model_saved(inputs_small)

In [9]:
assert (predictions == predictions_load).all()

In [10]:
# Log performance metric
mae = torch.mean(torch.abs(predictions - outputs_small)).item()

# Plot and save
nrows, ncols = 3, 2
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 15))
for i, output_name in enumerate(lume_module.output_order):
    ax_i = ax[i // ncols, i % ncols]
    if i < outputs_small.shape[1]:
        sort_idx = torch.argsort(outputs_small[:, i])
        x_axis = torch.arange(outputs_small.shape[0])
        ax_i.plot(x_axis, outputs_small[sort_idx, i], "C0x", label="outputs")
        ax_i.plot(x_axis, predictions[sort_idx, i], "C1x", label="predictions")
        ax_i.legend()
        ax_i.set_title(output_name)
ax[-1, -1].axis('off')
fig.tight_layout()

plot_path = "comparison_plot_lume.png"
plt.savefig(plot_path)
plt.close()

Note that the `lume_module.register_to_mlflow` ends the run automatically, but if you'd like to go back and update it, e.g. log an artifact, you can do so as follows:

In [11]:
run_id = model_info.run_id
with mlflow.start_run(run_id=run_id) as run:
    # log some metric
    mlflow.log_metric("mean_absolute_error", mae)
    # log the image file
    mlflow.log_artifact(plot_path)

🏃 View run lume-test at: http://127.0.0.1:8082/#/experiments/742261476526561220/runs/ce536dd7f16e48e69da0bd5ae8f20dbd
🧪 View experiment at: http://127.0.0.1:8082/#/experiments/742261476526561220
